In [ ]:
%%capture
# Remember to set runtime to GPU acceleration

# Mount files
from google.colab import drive
drive.mount('/content/drive')

# Set up Kaggle
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!mkdir ~/.kaggle

# tfa
!pip install tensorflow-addons

import json
token = {"username":"neilgoecknerwald","key":"KEY"}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# Download files
!kaggle competitions download -c tensorflow-great-barrier-reef

In [ ]:
%%capture
!unzip tensorflow-great-barrier-reef.zip
!rm tensorflow-great-barrier-reef.zip

In [ ]:
# Clone and pull in python files
!git config --global user.email "email@email.com"
!git config --global user.name "Neil Goeckner-Wald"

!git clone https://git@github.com/ngoecknerwald/starfish-perception-telescope.git

In [ ]:
# Boilerplate
import sys
import numpy as np
import tensorflow as tf
from matplotlib.pyplot import figure, imshow, gca, tight_layout, show
from matplotlib import patches
from importlib import reload
import os

if 'roi_pooling' not in sys.modules:
    import backbone
    import classifier
    import data_utils
    import faster_rcnn
    import rpn
    import roi_pooling
    import geometry
else:
    reload(backbone)
    reload(classifier)
    reload(data_utils)
    reload(faster_rcnn)
    reload(rpn)
    reload(roi_pooling)
    reload(geometry)

In [ ]:
# List physical devices
is_colab = len(tf.config.list_physical_devices('GPU')) > 0

# Data locations
if is_colab:
    datapath='/content'
    backbone_weights='drive/MyDrive/trained_backbone.ckpt'
    rpn_weights='drive/MyDrive/compiled_rpn.ckpt'
    class_weights='drive/MyDrive/compiled_classifier.ckpt'
else:
    datapath='tensorflow-great-barrier-reef'
    backbone_weights='trained_backbone.ckpt'
    rpn_weights='compiled_rpn.ckpt'
    class_weights='compiled_classifier.ckpt'

In [ ]:
# Instantiate the high-level wrapper
frcnn = faster_rcnn.FasterRCNNWrapper(
    input_shape=(720, 1280, 3),
    datapath=datapath,
    backbone_type='ResNet50',
    backbone_weights=backbone_weights if os.path.exists(backbone_weights) else 'finetune',
    rpn_weights=rpn_weights if os.path.exists(rpn_weights) else None,
    classifier_weights=class_weights if os.path.exists(class_weights) else None
)

In [ ]:
frcnn.rpnwrapper.save_rpn_state('drive/MyDrive/compiled_rpn.ckpt')

In [ ]:
# Test the RPN
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# Test that the RPN components work as expected
all_decoded = []
validation = frcnn.data_loader_full.get_training().__iter__()

# Find minibatch with a positive example
while all([np.sum(decoded) == 0 for decoded in all_decoded]):
    images, labels = validation.next()
    all_decoded = [frcnn.data_loader_full.decode_label(label) for label in labels]

# Run system in forward mode. First show the RoI to test
roi = frcnn.rpnwrapper.propose_regions(images, is_images=True)
roi = roi.astype('float32')

In [ ]:
# Plot everything up
roi = roi[:,:32,:]

for i, decoded in enumerate(all_decoded):

    figure(figsize=(16, 9))
    imshow(images[i, :, :, :].numpy() / 255.0)

    # Draw the ground truth
    for annotation in decoded:
        rect = patches.Rectangle(
            (annotation[0], annotation[1]),
            annotation[2],
            annotation[3],
            linewidth=4,
            edgecolor="y",
            facecolor="none",
        )
        gca().add_patch(rect)

    # Draw the proposals
    for j in range(roi.shape[1]):
        rect = patches.Rectangle(
            (roi[i, j, 0], roi[i, j, 1]),
            roi[i, j, 2],
            roi[i, j, 3],
            linewidth=np.maximum((roi.shape[1] - j) / 8, 1),
            edgecolor="g",
            facecolor="none",
        )
        gca().add_patch(rect)
    
    # Plot it up
    gca().grid("True")
    tight_layout()